In [9]:
import torch
from torch.utils.data import TensorDataset, DataLoader

torch.manual_seed(1)

train_X = torch.load("./train_X.pt")
train_Y = torch.load("./train_Y.pt")
valid_X = torch.load("./valid_X.pt")
valid_Y = torch.load("./valid_Y.pt")
test_X = torch.load("./test_X.pt")
test_Y = torch.load("./test_Y.pt")

train_data = TensorDataset(train_X, train_Y)
valid_data = TensorDataset(valid_X, valid_Y)

In [10]:
from torch import nn

class Perseptron(nn.Module):
  def __init__(self, input_size, output_size) -> None:
    super().__init__()

    # nn.Linear: 全結合層を作成する
    self.fc1 = nn.Linear(input_size, 100)
    self.fc2 = nn.Linear(100, output_size)

    # 重み行列を正規分布に上書き
    nn.init.normal_(self.fc1.weight, 0.0, 1.0)
    nn.init.normal_(self.fc2.weight, 0.0, 1.0)

  def forward(self, x):
    x = self.fc1(x)
    x = self.fc2(x)
    x = torch.softmax(x, dim=-1)
    return x

In [11]:
model = Perseptron(train_X.size()[1], 4)
loss_fn = nn.CrossEntropyLoss()

In [12]:
from torch import optim

op = optim.SGD(model.parameters(), lr=0.1)

In [13]:
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)

epochs = 250

for i in range(epochs):
    for j, (data, label) in enumerate(train_dataloader):
      pred = model(data)
      loss = loss_fn(pred, label)

      op.zero_grad()
      loss.backward()
      op.step()

In [14]:
pred = model(train_X)

accuracy = (pred.argmax(dim = 1) == train_Y).sum().item()
accuracy /= len(train_X)

accuracy

0.9150131037064769

In [15]:
pred = model(valid_X)

accuracy = (pred.argmax(dim = 1) == valid_Y).sum().item()
accuracy /= len(valid_X)

accuracy

0.8922155688622755

In [16]:
pred = model(test_X)

accuracy = (pred.argmax(dim = 1) == test_Y).sum().item()
accuracy /= len(test_X)

accuracy

0.8719101123595505